# Implent your own decision tree/random forest!


In this python notebook, you will create a basic decision tree on pandas data, and train a classifier on the Iris dataset. Then, you will implement a type of bagging and create a random forest classifier!

First, import the required modules:

In [14]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

Then import and preview the data:

In [16]:
iris = load_iris()
df = pd.DataFrame(iris.data)
df['species'] = iris.target 
df.head()

,0,1,2,3,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


We have four features labeled 0, 1, 2, and 3. I believe these stand for the length and the width of the sepals and petals, in centimeters. I don't quite know which is which in the dataset (or what a sepal is) but it doesn't really matter. We want to use these four features to predict whether the species is one of three types of Iris plant, labeled 0, 1, or 2. 

Now, we split the dataset into training and test samples.

In [20]:
df['is_train'] = np.random.uniform(0, 1, len(df)) <= .75
train, test = df[df['is_train']==True], df[df['is_train']==False]
train = train.drop(['is_train'], axis = 1)
test = test.drop(['is_train'], axis = 1)

First, we want to implement some measure of disorder in a set of data. Implement either entropy ($\sum_x -p(x)\log x$) or GINI impurity discussed in class. The argument `data` is a pandas dataframe containing the features and labels of several data points. We calculate disorder based on the labels, or the last column of the data. Note: make sure that you make this function work for different data (i.e. your function should work for data of different dimensions).

In [21]:
def disorder(data):
    counter = {}
    total = 0
    for x in data.iloc[:,data.shape[1]-1]:
        counter[x] = counter.get(x,0) + 1
    for count in counter.values():
        p = count/data.shape[0]
        total += -(p*np.log(p)/.693)
    return total

We now create a split function. This function takes in a dataset, and indices for a row and column. We then return two dataframes split on the `column`th feature. The left dataset should contain all of the data where the `column`th feature is greater or equal to the `column`th feature of the `row`th datapoint, and the right should contain the rest.

In [28]:
def split_on_row_column(data, row, column):
    left = data.loc[data.iloc[row, column] >= data.iloc[:, column]]
    right = data.loc[data.iloc[row, column] < data.iloc[:, column]]
    return left, right

We now want to define our recursive tree class. During training, there are two cases for a node. If the data is all one label, the node is a leaf node, and we return this value during inference. If the data is not all the same label, we find the best split of the data by iterating through all of features and rows in the data.

Inference takes in a row of a pandas dataframes and returns the predicted class. 

In [33]:
class Node:
    def __init__(self, data):
        self.data = data
    
    def train(self):
        if disorder(self.data) == 0:
            self.leaf = True
            self.value = self.data.iloc[0, self.data.shape[1] - 1]
            return
        self.leaf = False
        self.value = 420
        min_entropy = -1
        split_row = 0
        split_col = 0
        left = self.data
        right = self.data
        for col in range(self.data.shape[1] - 1):
            for row in range(self.data.shape[0]):
                l, r = split_on_row_column(self.data, row, col)
                e = disorder(l)*l.shape[0] + disorder(r)*r.shape[0]
                if min_entropy == -1 or e < min_entropy:
                    min_entropy = e
                    split_row = row
                    split_col = col
                    left = l
                    right = r
                    
        self.split_row = split_row
        self.split_col = split_col
        self.left = Node(left)
        self.right = Node(right)
        self.left.train()
        self.right.train()
            
   
        
    def inference(self, x):
        if self.leaf:
            return self.value
        if self.data.iloc[self.split_row, self.split_col] >= x.iloc[self.split_col]:
            return self.left.inference(x)
        return self.right.inference(x)

Now initialize and train a decision tree:

In [34]:
tree = Node(train)
tree.train()

Note that we don't check the training accuracy here (why?). We now want to validate our tree on the test dataset:

In [35]:
def validate(model, data):
    ct = 0
    corr = 0
    for i in range(test.shape[0]):
        data = test.iloc[i]
        ct += 1
        if model.inference(data) == data['species']:
            corr += 1
    return corr/ct

validate(tree, test)

0.9705882352941176

## Random Forest!

Now we will implement data bagging with a random forest! The set up is similar to a single tree. We pass in the data to the forest, along with hyperparameters `n` and `frac` which correspond to the number of trees and the fraction of the dataset to use in each bag. In the inference step we tally the number of votes from each decision tree and return the label with the most amount of votes.

In [36]:
class Forest:
    def __init__(self, data, n, frac):
        self.data = data
        self.n = n
        self.frac = frac
    
    def train(self):
        self.trees = []
        for i in range(self.n):
            print("Training tree", str(i))
            bag = self.data.sample(frac=self.frac)
            tree = Node(bag)
            tree.train()
            self.trees.append(tree)
    
    def inference(self, x):
        counts = dict()
        for tree in self.trees:
            y = tree.inference(x)
            counts[y] = counts.setdefault(y, 0) + 1
        
        maxCount = -1
        maxVal = 0
        
        for val, count in counts.items():
            if count > maxCount:
                maxCount = count
                maxVal = val
        return maxVal        

In [37]:
forest = Forest(train, 30, .5)
forest.train()

Training tree 0
Training tree 1
Training tree 2
Training tree 3
Training tree 4
Training tree 5
Training tree 6
Training tree 7
Training tree 8
Training tree 9
Training tree 10
Training tree 11
Training tree 12
Training tree 13
Training tree 14
Training tree 15
Training tree 16
Training tree 17
Training tree 18
Training tree 19
Training tree 20
Training tree 21
Training tree 22
Training tree 23
Training tree 24
Training tree 25
Training tree 26
Training tree 27
Training tree 28
Training tree 29


In [12]:
validate(forest, test)

0.9393939393939394